# Generic Rule Induction Notebook

Continuoulsy refined.

In [4]:
#%store -z acc_list 
#%store -z prec_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [5]:
from config import config_dict
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG-I4"]
#CONFIG = config_dict["CONFIG14"]

print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: german-credit-brcg


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *



# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [7]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Status Checking          1000 non-null   object
 1   Duration in Month        1000 non-null   int64 
 2   Credit History           1000 non-null   object
 3   Purpose                  1000 non-null   object
 4   Credit Amount            1000 non-null   int64 
 5   Savings Account          1000 non-null   object
 6   Employement since        1000 non-null   object
 7   Installmentrate %        1000 non-null   int64 
 8   StatusSex                1000 non-null   object
 9   Otherdebtos              1000 non-null   object
 10  PresentResidence         1000 non-null   int64 
 11  Property                 1000 non-null   object
 12  Age in years             1000 non-null   int64 
 13  Otherinstallment Plans   1000 non-null   object
 14  Housing                  1000 non-null   

0    700
1    300
Name: Target, dtype: int64

In [8]:


#if CONFIG['UNDERSAMP'] == True:
#    under_sampling_fraud = df[df[CONFIG['TARGET_LABEL']] == 1]
#    under_size = under_sampling_fraud.size
#    under_sampling_legit = (df[df[CONFIG['TARGET_LABEL']] == 0]).sample(n=10000, random_state = 42)
#    df = pd.concat([under_sampling_legit, under_sampling_fraud])

In [9]:
CONFIG['POS_CLASS']

1

In [10]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    700
1    300
Name: Target, dtype: int64


### Train, Test Split

In [11]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (700, 20) (700,)
Test: (300, 20) (300,)


### Reference Performance

In [12]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [13]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 541 to 102
Data columns (total 28 columns):
 #   Column                                                                              Non-Null Count  Dtype
---  ------                                                                              --------------  -----
 0   (Age in years, <=, 23.5)                                                            700 non-null    int32
 1   (Age in years, >, 23.5)                                                             700 non-null    int32
 2   (Credit Amount, <=, 7897.0)                                                         700 non-null    int32
 3   (Credit Amount, <=, 10975.5)                                                        700 non-null    int32
 4   (Credit Amount, >, 7897.0)                                                          700 non-null    int32
 5   (Credit Amount, >, 10975.5)                                                         700 non-null    int32
 6   

feature   Age in years      Credit Amount                         \
operation           <=    >            <=              >           
value             23.5 23.5        7897.0 10975.5 7897.0 10975.5   
541                  0    1             1       1      0       0   
440                  0    1             1       1      0       0   
482                  0    1             1       1      0       0   
422                  0    1             1       1      0       0   
778                  0    1             1       1      0       0   

feature                                                 Credit History  \
operation                                                           !=   
value     critical account / other credits existing (not at this bank)   
541                                                        1             
440                                                        1             
482                                                        1             
422                                                        0             
778                                                        0             

feature                                                                 \
operation                                                           ==   
value     critical account / other credits existing (not at this bank)   
541                                                        0             
440                                                        0             
482                                                        0             
422                                                        1             
778                                                        1             

feature   Duration in Month       ... PresentResidence       Purpose  \
operation                <=    >  ...                >            !=   
value                  33.0 33.0  ...              2.5 3.5 car (new)   
541                       1    0  ...                1   1         0   
440                       1    0  ...                1   1         0   
482                       1    0  ...                1   1         1   
422                       1    0  ...                1   0         0   
778                       0    1  ...                0   0         1   

feature                                               Savings Account          \
operation                         ==                               !=      ==   
value     radio/television car (new) radio/television         < 100DM < 100DM   
541                      1         1                0               0       1   
440                      1         1                0               0       1   
482                      1         0                0               1       0   
422                      1         1                0               0       1   
778                      1         0                0               1       0   

feature   Status Checking      
operation              !=  ==  
value                 A14 A14  
541                     0   1  
440                     0   1  
482                     1   0  
422                     1   0  
778                     0   1  

[5 rows x 28 columns]

### Rule Induction

In [14]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                      # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.2906
Iteration: 2, Objective: 0.2443
Iteration: 3, Objective: 0.2387
Iteration: 4, Objective: 0.2354
Iteration: 5, Objective: 0.2354
Training time: 3.9186015129089355


In [15]:
CONFIG['POS_CLASS']

1

### Evaluation

In [16]:
acc_list = []
prec_list = []
%store -r acc_list
%store -r prec_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    prec_list.append(precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    %store acc_list
    %store prec_list
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.69
Balanced accuracy: 0.5541563699458436
Precision: 0.475
Recall: 0.2087912087912088
F1 0.2900763358778626
ConfusionMatrix [[188  21]
 [ 72  19]]
F-2 0.23514851485148514
Mathhews 0.14647270820114072
Stored 'acc_list' (list)
Stored 'prec_list' (list)


In [17]:
y_test.value_counts()

0    209
1     91
Name: Target, dtype: int64

In [18]:
CONFIG['POS_CLASS']

1

In [19]:
CONFIG['BINARIZER']

'TREES'

In [20]:
CONFIG['ALGO']

'BRCG'

In [21]:
df[CONFIG["TARGET_LABEL"]].value_counts()

0    700
1    300
Name: Target, dtype: int64

In [22]:
CONFIG['POS_CLASS']

1

In [23]:
präds = []
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    #print(estimator.rule_list_to_pretty_string())
    # Get predicates --> if Key Error Switch 0 to 1 or 1 to 0
    for i in range(len(estimator.rule_map[1])):
        print(len(estimator.rule_map[1][i]))
        präds.append(len(estimator.rule_map[1][i]))
    print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    # Get predicates
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        präds.append(len(estimator.rl().rules[i]["antecedents"]))
        print(f"Antecedents Length Rule {i}:" , an)
    print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 4
['Credit Amount > 10975.50 AND Duration in Month <= 33.00', 'Credit Amount > 7897.00 AND Job != unskilled-resident AND Savings Account == < 100DM AND Status Checking != A14', 'Credit Amount <= 7897.00 AND Credit History != critical account / other credits existing (not at this bank) AND Purpose == car (new) AND Status Checking != A14', 'Credit History != critical account / other credits existing (not at this bank) AND Duration in Month > 33.00 AND Employement since != unemployed AND PresentResidence > 2.50 AND Purpose != radio/television AND Status Checking != A14']
